In [1]:
import os

In [3]:
%pwd

'c:\\Users\\rupan\\Documents\\Projects\\Deep Learning\\Chicken-Disease-Classification\\research'

In [4]:
os.chdir("../")
%pwd

'c:\\Users\\rupan\\Documents\\Projects\\Deep Learning\\Chicken-Disease-Classification'

In [35]:
# update the entity

from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [30]:
# update config manager in src

# from CNNClasl(PARAsifier.constants import *
# from CNNClassifier.utils.common import read_yaml, create_directories

# config = read_yaml(CONFIG_FILE_PATH)
# params = read_yamMS_FILE_PATH)

In [31]:
# config = config.prepare_base_model

# # create root directory
# create_directories([config.root_dir])

# base_model_config = PrepareBaseModelConfig(
#     root_dir=config.root_dir,
#     base_model_path=config.base_model_path,
#     updated_base_model_path=config.updated_base_model_path,
#     params_image_size=params.IMAGE_SIZE,
#     params_learning_rate=params.LEARNING_RATE,
#     parans_include_top=params.INCLUDE_TOP,
#     params_weights=params.WEIGHTS,
#     params_classes=params.CLASSES
# )

In [36]:
# update configuration manager in src

from CNNClassifier.constants import *
from CNNClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:

    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
        root_dir=Path(config.root_dir),
        base_model_path=Path(config.base_model_path),
        updated_base_model_path=Path(config.updated_base_model_path),
        params_image_size=self.params.IMAGE_SIZE,
        params_learning_rate=self.params.LEARNING_RATE,
        params_include_top=self.params.INCLUDE_TOP,
        params_weights=self.params.WEIGHTS,
        params_classes=self.params.CLASSES
        )
        return prepare_base_model_config
        

In [70]:
# create component

import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf


class PrepareBaseModel:

    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape=self.config.params_image_size,
            weights=self.config.params_weights,
            include_top=self.config.params_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_all is not None) and (freeze_till > 0):
            for layer in model.layers[: -freeze_till]:
                model.trainable = False
        
        flatten_in = tf.keras.layers.Flatten()(model.output)
        prediction = tf.keras.layers.Dense(units=classes,activation="softmax")(flatten_in)

        full_model = tf.keras.models.Model(inputs=model.input,outputs=prediction)

        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        
        full_model.summary()
        return full_model
    
    def update_base_model(self):
        self.full_model = self.prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)
    

In [86]:
model = prepare_base_model.model
config = prepare_base_model.config

In [81]:
def prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
    if freeze_all:
        for layer in model.layers:
            model.trainable = False
    elif (freeze_all is not None) and (freeze_till > 0):
        for layer in model.layers[: -freeze_till]:
            model.trainable = False
    
    flatten_in = tf.keras.layers.Flatten()(model.output)
    prediction = tf.keras.layers.Dense(units=classes,activation="softmax")(flatten_in)

    full_model = tf.keras.models.Model(inputs=model.input,outputs=prediction)

    full_model.compile(
        optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
    )
    
    # full_model.summary()
    return full_model


full_model = prepare_full_model(
            model=model,
            classes=config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=config.params_learning_rate
        )


In [85]:
# save_model(path=config.updated_base_model_path, model=full_model)
full_model.save(config.updated_base_model_path)

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\prepare_base_mode\\base_model_updated.keras'

In [87]:
config.updated_base_model_path

WindowsPath('artifacts/prepare_base_mode/base_model_updated.keras')

In [57]:
flatten_in = tf.keras.layers.Flatten()(model.output)
prediction = tf.keras.layers.Dense(units=config.params_classes,activation="softmax")(flatten_in)

In [69]:
full_model = tf.keras.models.Model(inputs=model.input,outputs=prediction)
full_model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=config.params_learning_rate),
        loss=tf.keras.losses.CategoricalCrossentropy(),
        metrics=["accuracy"]
    )

In [78]:

# pipeline

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e


[2024-06-26 21:09:54,016: INFO : common: yaml file: config\config.yaml loaded successfully]
[2024-06-26 21:09:54,020: INFO : common: yaml file: params.yaml loaded successfully]
[2024-06-26 21:09:54,022: INFO : common: created directory at: artifacts]
[2024-06-26 21:09:54,023: INFO : common: created directory at: artifacts/prepare_base_model]


Model: "functional_11"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)      │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 224, 224, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 224, 224, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 112, 112, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 112, 112, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 112, 112, 128)  │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 56, 56, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 56, 56, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 56, 56, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 28, 28, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 28, 28, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 28, 28, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 14, 14, 512)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 14, 14, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 7, 7, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_12 (Flatten)            │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 2)              │        50,178 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,764,866 (56.32 MB)

 Trainable params: 50,178 (196.01 KB)

 Non-trainable params: 14,714,688 (56.13 MB)

FileNotFoundError: [Errno 2] No such file or directory: 'artifacts\\prepare_base_mode\\base_model_updated.keras'